In [ ]:
import numpy as np
from os import listdir

def img2vector(directory, filename):
    vector1 = np.zeros((1, 1024))
    vector2 = np.zeros((1, 256))
    data = np.zeros((32, 32))
    f = open(directory + '/' + filename)
    for i in range(32):
        line = f.readline()
        for j in range(32):
            data[i, j] = int(line[j])
            vector1[0, 32*i+j] = int(line[j])
    for i in range(0, 32, 2):
        for j in range(0, 32, 2):
            vector2[0, int(i*8+j/2)] = data[i][j]+data[i][j+1]+data[i+1][j]+data[i+1][j+1]
    return vector1, vector2

def getData(filename):
    fileList = listdir(filename)
    m = len(fileList)
    dataset = np.zeros((m, 1024))
    dataset_DR = np.zeros((m, 256))
    labels = []
    for i in range(m):
        labels.append(int(fileList[i].split('_')[0]))
        dataset[i], dataset_DR[i] = img2vector(filename, fileList[i])
    return dataset, dataset_DR, labels

In [ ]:
from sklearn.linear_model import Ridge
np.set_printoptions(threshold=np.inf)

if __name__ == '__main__':
    X_train, X_train_DR, y_train = getData('trainingDigits')
    X_test, X_test_DR, y_test = getData('testDigits')
    reg = Ridge()
    reg = reg.fit(X_train, y_train)
    result = reg.score(X_test, y_test)
    print(result)
    X_train, X_train_DR, y_train = getData('trainingDigits')
    X_test, X_test_DR, y_test = getData('testDigits')
    reg = reg.fit(X_train, y_train)
    result = reg.score(X_test, y_test)
    print(result)